# Notebook 3 : Apprentissage non-supervisé

Notebook préparé par [Chloé-Agathe Azencott](http://cazencott.info).

Dans ce notebook il s'agit d'explorer plusieurs techniques de réduction de dimension et de clustering.

In [ ]:
# charger numpy as np, matplotlib as plt
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rc('font', **{'size': 12}) # règle la taille de police globalement pour les plots (en pt)

In [ ]:
import pandas as pd

## 1. Analyse en composantes principales

Dans cette section, nous allons effectuer une analyse en composantes principales d'un jeu de données décrivant les scores obtenus par les meilleurs athlètes ayant participé en 2004 à une épreuve de décathlon, aux Jeux Olympiques d'Athènes ou au Décastar de Talence.

### Chargement des données

Les données sont contenues dans le fichier `decathlon.txt`.

Le fichier contient 42 lignes et 13 colonnes.

La première ligne est un en-tête qui décrit les contenus des colonnes.

Les lignes suivantes décrivent les 41 athlètes.

Les 10 premières colonnes contiennent les scores obtenus aux différentes épreuves.
La 11ème colonne contient le classement.
La 12ème colonne contient le nombre de points obtenus.
La 13ème colonne contient une variable qualitative qui précise l'épreuve (JO ou Décastar) concernée.

Nous allons examiner ces données en commençant avec la librairie `pandas`.

In [ ]:
my_data = pd.read_csv('data/decathlon.txt', sep="\t")  # lire les données dans un dataframe

__Alternativement :__ Si vous avez besoin de télécharger le fichier (par exemple sur colab) décommentez les deux lignes suivantes :

In [ ]:
# !wget https://raw.githubusercontent.com/CBIO-mines/fml-dassault-systems/main/data/decathlon.txt

# my_data = pd.read_csv('decathlon.txt', sep="\t") 

In [ ]:
my_data.head()

### Visualisation

Une __matrice de nuages de points__ est une visualisation en k x k panneaux des relations deux à deux entre k variables :
* sur la diagonale, l'histogramme pour chacune des variables 
* hors diagonale, les nuages de points entre deux variables (non standardisées).

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.plotting.scatter_matrix.html

A vous d'afficher la visualisation à l'aide de la fonction `scatter_matrix` : 

In [ ]:
from pandas.plotting import scatter_matrix

### DEBUT DE VOTRE CODE
...
### FIN DE VOTRE CODE

Vous pouvez aussi limiter la visualisation à quelques variables, pour des observations plus claires. Affichez la scatter_matrix pour les 3 ou 4 variables qui vous semblent les plus corrélées par exemple : 

In [ ]:
### DEBUT DE VOTRE CODE
...
### FIN DE VOTRE CODE

Alternativement, la librairie `seaborn` permet des visualisations plus élaborées que `matplotlib`. Vous pouvez par exemple explorer les capacités de `jointplot`. 
https://seaborn.pydata.org/generated/seaborn.jointplot.html

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')

sns.jointplot(x='Shot.put', y='400m', data = my_data, 
              height=6, space=0, color='b')

sns.jointplot(x='Rank', y='Points', data = my_data, 
              kind='reg', height=6, space=0, color='b');

Nous allons maintenant effectuer une analyse en composantes principales des scores aux 10 épreuves.

Commençons par extraire les variables prédictives :

In [ ]:
X = np.array(my_data.drop(columns=['Points', 'Rank', 'Competition']))
print(X.shape)

### Standardisation des données

Après visualisation des données, on peut remarquer des échelles et des distributions de données différentes selon les variables. 
On réapplique donc ici la procédure vue dans les TPs précédents pour standardiser nos données :  nous avons besoin d'un objet `StandardScaler` contenu dans le module `preprocessing` de `sklearn`

In [ ]:
### DEBUT DE VOTRE CODE
# Import du module
...

In [ ]:
# Creation de l'objet StandardScaler
std_scaler = ...

# Ajustement de l'objet sur les données
...

# Transformation des données
X_scaled = ...

### FIN DE VOTRE CODE

print(X_scaled.shape)

### Calcul des composantes principales

Les algorithmes de factorisation de matrice de `scikit-learn` sont inclus dans le module `decomposition`. Pour  l'ACP, référez-vous à : 
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
from sklearn import decomposition

Remarque : Nous avons ici peu de variables et pouvons nous permettre de calculer toutes les PC. 

La plupart des algorithmes implémentés dans `scikit-learn` suivent le fonctionnement suivant : 
* on instancie un objet, correspondant à un type d'algorithme/modèle, avec ses hyperparamètres (ici le nombre de composantes)
* on utilise la méthode `fit` pour passer les données à cet algorithme
* les paramètres appris sont maintenant accessibles comme arguments de cet objet.

In [ ]:
# Instanciation d'un objet PCA pour 10 composantes principales

### DEBUT DE VOTRE CODE
pca = ...

In [ ]:
# On passe maintenant les données standardisées à cet objet
# C'est ici que se font les calculs
...

### FIN DE VOTRE CODE

### Proportion de variance expliquée par les PCs

Nous allons maintenant afficher la proportion de variance expliquée par les différentes composantes. Il est accessible dans le paramètre `explained_variance_ratio_` de notre objet `pca`.

In [ ]:
plt.plot(np.arange(1, 11), pca.explained_variance_ratio_, marker='o')

plt.xlabel("Nombre de composantes principales")
plt.ylabel("Proportion de variance expliquée")
plt.show()

Nous pouvons aussi afficher la proportion *cumulative* de variance expliquée, avec la fonction [`cumsum`](https://numpy.org/doc/2.1/reference/generated/numpy.cumsum.html) de `numpy` (importé plus haut sous l'alias `np`)

Affichez sur un graphique similaire à celui ci-dessus, la proportion cumulative de variance expliquée en fonction du nombre de composantes principales considérées

In [ ]:
### DEBUT DE VOTRE CODE
...
### FIN DE VOTRE CODE

__Questions :__ 
* Quelle est la proportion de variance expliquée par les deux premières composantes ? 
* Combien de composantes faudrait-il utiliser pour expliquer 80% de la variance des données ?

### Projection des données sur les deux premières composantes principales

Nous allons maintenant utiliser uniquement les deux premières composantes principales.

Commençons par calculer la nouvelle représentation des données, c'est-à-dire leur projection sur ces deux PC. 

In [ ]:
X_projected = pca.transform(X_scaled)
print(X_projected.shape)

On peut afficher un nuage de points représentant les données selon ces deux PC.

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(X_projected[:, 0], X_projected[:, 1])

plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.show()

On peut maintenant colorer chaque point du nuage de points ci-dessus en fonction du classement de l'athlète qu'il représente. 

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(X_projected[:, 0], X_projected[:, 1], c=my_data['Rank'])

plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.colorbar(label='classement')
plt.show()

__Question :__ Qu'en conclure sur l'interprétation de la PC1 ?

### Interprétation des deux premières composantes principales
Chaque composante principale est une combinaison linéaire des variables décrivant les données. Les poids de cette combinaison linéaire sont accesibles dans `pca.components_`.

Nous pouvons maintenant visualiser non pas les individus comme ci-dessus, mais les 10 variables dans l'espace des 2 composantes principales.

In [ ]:
pcs = pca.components_
print(pcs[0])

In [ ]:
fig = plt.figure(figsize=(6, 6))

plt.scatter(pcs[0], pcs[1])
for (x_coordinate, y_coordinate, feature_name) in zip(pcs[0], pcs[1], my_data.columns[:10]):
    plt.text(x_coordinate, y_coordinate, feature_name)                          
    
plt.xlabel("Contribution à la PC1")
plt.ylabel("Contribution à la PC2")
plt.show()

__Question :__ Quelles variables ont des contributions très similaires aux deux composantes principales ? Qu'en déduire sur leur similarité ?

__Question :__ Comment interpréter le signe des contributions des variables à la première composantes principales ?

## 2. Données « Olivetti »

Nous allons maintenant utiliser la réduction de dimension pour représenter en deux dimensions un jeu de données contenant des visages. Il s'agit d'un jeu de données classique, contenant 400 photos de 64 par 64 pixels. Il s'agit de photos des visages de 40 personnes différentes (10 photos par personne), étiquetées par un numéro de classe entre 0 et 39 identifiant la personne.

Nous pouvons charger ce jeu de données directement grâce à scikit-learn :

In [ ]:
from sklearn import datasets

In [ ]:
data = datasets.fetch_olivetti_faces()

__Si vous n'arrivez pas à télécharger les données :__
* Aller sur : https://github.com/CroncLee/PCA-face-recognition/blob/master/olivetti_py3.pkz
* Télécharger le fichier (bouton Download) 
utiliser la commande
```
    data = datasets.fetch_olivetti_faces(data_home="<PATH TO DATA>")
```
En remplaçant <PATH TO DATA> par le chemin vers le dossier où vous avez enregistré les données.

In [ ]:
X = data.data
y = data.target

In [ ]:
print(X.shape)

In [ ]:
print("Les données contiennent %d classes" % len(np.unique(y)))

Chaque image est représentée par une valeur (niveau de gris) pour chacun de ses pixels. 

Nous pouvons visualiser ces images à condition de réorganiser ces valeurs (= un vecteur de longueur 4096) en matrices 64x64. Par exemle ci-dessous pour l'image à l'index 23 :

In [ ]:
plt.imshow(X[77, :].reshape((64, 64)), interpolation='nearest', cmap=plt.cm.gray)

### PCA

Commençons par une analyse en composantes principales comme à la section précédente :

In [ ]:
pca = decomposition.PCA(n_components=2)
X_transformed_pca = pca.fit_transform(X)

Chaque image est maintenant représentée par non pas 4096 variables, mais par deux variables. Nous pouvons les visualiser en nuage de point, et les colorer par classe :

In [ ]:
plt.scatter(X_transformed_pca[:, 0], X_transformed_pca[:, 1], c=y)
plt.xlabel("Première composante principale")
plt.ylabel("Deuxième composante principale")
plt.show()

__Question :__ les images du même visage (= de la même classe) ont-elles des représentations proches ?

Nous pouvons visualiser la contribution de chaque pixel à la première composante principale :

In [ ]:
plt.imshow(pca.components_[0, :].reshape((64,64)), interpolation='nearest', cmap=plt.cm.gray);

Puis la contribution de chaque pixel à la deuxième composante principale :

In [ ]:
plt.imshow(pca.components_[1, :].reshape((64,64)), interpolation='nearest', cmap=plt.cm.gray);

**Question :** Quelle interprétation pouvons nous faire à partir de ces deux images représentant les contributions de chaque pixel sur les deux composantes principales ?

### tSNE

Essayons une autre méthode de réduction de la dimensionnalité pour tenter de mieux séparer nos classes

Nous allons maintenant utiliser la même démarche que pour la PCA, mais avec l'algorithme tSNE, grâce à la classe [TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) du module `manifold`.

A vous :
- de créer l'objet `tsne` à partir de la classe citée ci-dessus. On veut ici ramener notre jeu de données en deux dimensions pour le visualiser
- d'intégrer les informations de notre jeu de données à cet objet
- de transformer nos données pour obtenir leurs nouvelles coordonnées en deux dimensions

In [ ]:
from sklearn import manifold

In [ ]:
### DEBUT DE VOTRE CODE
tsne = ...
X_transformed = ...
### FIN DE VOTRE CODE

Affichons le résultat de la réduction de dimensions avec tSNE :

In [ ]:
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], c=y)
plt.xlabel("Première composante tSNE")
plt.ylabel("Deuxième composante tSNE");

#### Influence du paramètre de perplexité 

Le principal hyperparamètre influant sur la représentation obtenue par l'algorithme tSNE est le paramètre de perplexité. Celui-ci représente le nombre de voisins pour lesquels les distances sont préservées. Cela a donc une influence sur la conservation de la structure locale (perplexité faible) ou globale (perplexité élevée). La représentation obtenue peut varier très fortement en fonction de ce paramètre.

Testez différentes valeurs du paramètre de perplexité et affichez les résultats correspondants

In [ ]:
### DEBUT DE VOTRE CODE
tsne_low_perp = ...
X_transformed_low_perp = ...
### FIN DE VOTRE CODE

In [ ]:
plt.scatter(X_transformed_low_perp[:, 0], X_transformed_low_perp[:, 1], c=y)
plt.xlabel("Première composante tSNE")
plt.ylabel("Deuxième composante tSNE")
plt.title("tSNE (faible perplexité)")

In [ ]:
### DEBUT DE VOTRE CODE
tsne_high_perp = ...
X_transformed_high_perp = ...
### FIN DE VOTRE CODE

In [ ]:
plt.scatter(X_transformed_high_perp[:, 0], X_transformed_high_perp[:, 1], c=y)
plt.xlabel("Première composante tSNE")
plt.ylabel("Deuxième composante tSNE")
plt.title("tSNE (perplexité élevée)")

## 3. Clustering

Générons trois jeux de données en deux dimensions:
- quatre clusters séparés issus de distributions normales
- deux demi-cercles imbriqués (ou "demi-lunes")
- deux cercles concentriques

In [ ]:
from sklearn import datasets

In [ ]:
# nombre de points
n_samples = 1000

# set random seed
np.random.seed(23)

four_blobs, four_blobs_labels = datasets.make_blobs(n_samples=n_samples, centers=4, n_features=2, random_state=170)

moons, moons_labels = datasets.make_moons(n_samples=n_samples, noise=0.05, random_state=170)

circles, circles_labels = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05, random_state=170)

Visualisons ces données :

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 3))

ax[0].scatter(four_blobs[:, 0], four_blobs[:, 1], c=four_blobs_labels, s=20, alpha=0.7)
ax[1].scatter(moons[:, 0], moons[:, 1], c=moons_labels, s=20, alpha=0.7)
ax[2].scatter(circles[:, 0], circles[:, 1], c=circles_labels, s=20, alpha=0.7)

ax[0].set_title('4 clusters')
ax[1].set_title('Lunes')
ax[2].set_title('Cercles')

plt.show()

Supposons maintenant que nous ne disposons **pas** des étiquettes. Quels algorithmes de clustering permettent de retrouver les clusters correspondant respectivement aux quatre blobs, deux lunes et deux cercles ? 

### Algorithme des k-moyennes

L'objectif de l'algorithme k-means est de retrouver $K$ clusters (et leur centroïde $\mu_k$) de manière à **minimiser la variance intra-cluster** :

\begin{align}
V = \sum_{k = 1}^{K} \sum_{x \in C_k} \frac{1}{|C_k|} (\|x - \mu_k\|^2)
\end{align}

**Implémentation à la main**

Nous allons commencer par implémenter l'algorithme "à la main", étape par étape, pour bien comprendre et visualiser ce que fait l'algorithme. Nous verrons ensuite comment utiliser directement l'implémentation du K-means dans la librairie `sklearn`. Les différentes étapes de l'algorithme sont : 
1. Sélectionner un nombre `k` de clusters (hyperparamètre)
2. Initialiser aléatoirement les `k` centroïdes parmi nos points de données
3. Calculer les distances de tous les points à ces centroïdes
4. Assigner à chaque point le cluster du centroïde le plus proche
5. Calculer la position des nouveaux centroïdes
6. Répéter les étapes 3 à 5 jusqu'à convergence, c'est-à-dire jusqu'à ce que les centroïdes ne changent plus d'une itération à l'autre

Essayons donc cet algorithme sur le dataset des 4 blobs, en commençant par choisir `k` et en prenant `k` points au hasard dans notre dataset qui constitueront les centroïdes initiaux :

In [ ]:
# VOTRE CHOIX DE PARAMETRE
k = ...

random_indices = np.random.choice(len(four_blobs), k, replace=False)
centroids_step0 = four_blobs[random_indices]

for i, centroid in enumerate(centroids_step0):
    print(f"Centroïde {i} : x = {centroid[0]}, y = {centroid[1]}")

Visualisons les données avec ces centroïdes initiaux

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

ax.scatter(four_blobs[:, 0], four_blobs[:, 1], c='grey', s=20, alpha=0.7)
ax.scatter(centroids_step0[:, 0], centroids_step0[:, 1], c='red', marker='x')

plt.show()

Calculons maintenant les distances de chaque point de donnée à ces centroïdes

In [ ]:
# Euclidean distance between data point and centroid
def compute_distance(data_point, centroid):
    dist = np.sqrt(np.sum((data_point - centroid)**2))
    return dist

def compute_all_distances(dataset, centroids):
    # Initialize distances array
    distances = np.zeros((k, n_samples))  # k and n_samples defined above
    
    # DEBUT DE VOTRE CODE

    # Calculate distance from each point to each centroid
    for i in ...:
        for j in ...:
            distances[i, j] = ...

    # FIN DE VOTRE CODE

    return distances

In [ ]:
distances = compute_all_distances(four_blobs, centroids_step0)

# Exemple de distance des 5 premiers points aux k centroïdes
print("Distances :")
print("\t\t", " \t\t".join([f"Point {i+1}" for i in range(5)]))
for i in range(k):
    print(f"Centroïde {i}\t", "\t".join(distances[i, :5].astype(str).tolist()))

Nous devons maintenant assigner à chaque point le cluster correspondant au centroïde le plus proche. On utilise pour ça la fonction `argmin` de `numpy`. Cela constitue en quelques sortes nos étiquettes prédites de façon intermédiaire.

In [ ]:
def assign_cluster(distances):
    assignments = np.argmin(distances, axis=0)
    return assignments

intermediate_labels = assign_cluster(distances)
print(intermediate_labels[:5])  # exemples d'étiquettes intermédiaires assignées aux points de données

On peut d'ailleurs vérifier que les labels intermédiaires assignés correspondent bien au centroïde le plus proche en comparant aux distances calculées plus haut pour ces 5 points.

Nous allons maintenant visualiser ces clusters intermédiaires et les centroïdes initiaux sur un scatter plot  

In [ ]:
def visualise_kmeans(dataset, labels, centroids, ax):

    ax.scatter(dataset[:, 0], dataset[:, 1], c=labels, s=20, alpha=0.7)
    ax.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x')
    
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
visualise_kmeans(four_blobs, intermediate_labels, centroids_step0, ax)
plt.show()


Il faut maintenant calculer la position des nouveaux centroïdes, que l'on va afficher sur le graphique

In [ ]:
def compute_new_centroids(dataset, labels):
    centroids = np.zeros((k, dataset.shape[1]))

    for i in range(k):
        centroids[i] = dataset[labels == i].mean(axis=0)
    
    return centroids

centroids_step1 = compute_new_centroids(four_blobs, intermediate_labels)

fig, axes = plt.subplots(1, 2, figsize=(9, 4))
visualise_kmeans(four_blobs, intermediate_labels, centroids_step0, axes[0])
visualise_kmeans(four_blobs, intermediate_labels, centroids_step1, axes[1])

axes[0].set_title("Centroïdes initiaux")
axes[1].set_title("Centroïdes après une itération")

plt.show()

On recommence maintenant les différentes étapes : calcul de distance aux centroïdes, assignation des clusters aux points de données et calcul des nouveaux centroïdes

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(24, 4))
n_iter = 10  # Nombre d'itérations de l'algorithme

current_fig = 0

centroids = centroids_step1
for i in range(n_iter):
    
    # DEBUT DE VOTRE CODE

    distances = ...
    intermediate_labels = ...

    # On affiche la visualisation toutes les 2 itérations
    if (i+1) % 2 == 0:
        visualise_kmeans(four_blobs, intermediate_labels, centroids, axes[current_fig])
        axes[current_fig].set_title(f"Itération {i+1}")
        current_fig += 1
        
    centroids = ...

    # FIN DE VOTRE CODE

plt.show()


On voit bien ici comment, par itérations successives, l'algorithme est capable d'identifier correctement nos clusters, en déplaçant petit à petit les centroïdes et en réajustant l'appartenance de nos points de données. 

**Question** : Dans quel(s) cas l'algorithme peut-il donner de mauvais résultats ?

**Implémentation avec `sklearn`**

Nous allons maintenant reprendre les 3 datasets créés plus haut (4 clusters, Demi-lunes et cercles concentriques), et leur appliquer l'algorithme implémenté dans la librairie sklearn, qui reprend les étapes que nous venons de voir.

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
from sklearn import cluster

In [ ]:
# DEBUT DE VOTRE CODE
# Initialisation de trois modèles k-means avec le nombre
# de clusters théoriquement attendu (resp. 4, 2 et 2):
kmeans_four_blobs = ...
kmeans_moons = ...
kmeans_circles = ...

# Application aux données 
...
...
...
### FIN DE VOTRE CODE

L'attribut `.labels_` contient, pour chaque observation, le numéro du cluster auquel cette observation est assignée.

In [ ]:
kmeans_four_blobs.labels_

Voyons maintenant à quoi ressemble le clustering obtenu pour nos trois datasets :

In [ ]:
# Visualisation du clustering
fig, ax = plt.subplots(1, 3, figsize=(10, 3))

ax[0].scatter(four_blobs[:, 0], four_blobs[:, 1], c=kmeans_four_blobs.labels_)
ax[1].scatter(moons[:, 0], moons[:, 1], c=kmeans_moons.labels_)
ax[2].scatter(circles[:, 0], circles[:, 1], c=kmeans_circles.labels_)

ax[0].set_title('4 clusters (k=4)')
ax[1].set_title('Lunes (k=2)')
ax[2].set_title('Cercles (k=2)')

**Questions :** Est-ce le clustering espéré ? Dans quels cas l'algorithme des k-moyennes fonctionne-t-il correctement ? Pourquoi ne fonctionne-t-il pas dans les autres cas ?

#### Trouver $K$ avec le coefficient de silhouette

Souvent, le nombre de clusters exact, $K$, n'est pas connu à l'avance. Nous pouvons tout de même appliquer l'algorithme k-means et mesurer la performance du clustering pour trouver le meilleur paramètre $K$. L'une des métriques utilisées est le **coefficient de silhouette**.

Le coefficient (ou score) de silhouette permet de **comparer les distances moyennes intra- et inter-cluster** :

\begin{align}
\text{score} = \frac{b - a}{\max(a, b)}
\end{align}

avec $a$ la distance moyenne intra-cluster et $b$ la distance d'un point au cluster étranger le plus proche. Le score se calcule par observation (avec une valeur entre -1 et 1) puis la moyenne de ce score permet d'évaluer le clustering du nuage de point dans son ensemble.

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score

In [ ]:
from sklearn import metrics

In [ ]:
print(f"4 clusters: Coefficient de silhouette pour le k-means (k=4) : %.2f" % 
      metrics.silhouette_score(four_blobs, kmeans_four_blobs.labels_))
print(f"Lunes: Coefficient de silhouette pour le k-means (k=2) : %.2f" % 
      metrics.silhouette_score(moons, kmeans_moons.labels_))
print(f"Cercles: Coefficient de silhouette pour le k-means (k=2) : %.2f" % 
      metrics.silhouette_score(circles, kmeans_circles.labels_))


Essayons d'évaluer la performance du clustering en fonction du nombre $K$ de clusters, en le faisant varier dans la fourchette [2, .., 8]

In [ ]:
k_values = range(2, 9)
names = ['4 clusters', 'Lunes', 'Cercles']
fig, ax = plt.subplots(2, 3, figsize=(16, 10))

for i, dataset in enumerate([four_blobs, moons, circles]):
    silhouettes = []
    
    for kval in k_values:

        ### DEBUT DE VOTRE CODE
        # Initialisez un modèle KMeans avec le nombre de clusters testé:
        kmeans_k = ...
        
        # Entraînez le modèle sur les données
        ...
        
        # Ajoutez le score de silhouette obtenu à la liste
        ...
        
        ### FIN DE VOTRE CODE
    
    # Visualisation du score de silhouette
    ax[0,i].plot(k_values, silhouettes)
    ax[0,i].set_xlabel("K")
    ax[0,i].set_ylabel("Silhouette score")
    ax[0,i].set_title(names[i])
    
    print("Dataset:", names[i])
    best_silhouette = np.max(silhouettes)
    print("Coefficient de Silhouette optimal : %.2f" % best_silhouette)
    best_K = k_values[silhouettes.index(best_silhouette)]
    print("Nombre de clusters K correspondant: %.0f" % best_K)
    
    
    kmeans_k = cluster.KMeans(n_clusters=best_K)
    kmeans_k.fit(dataset)
    ax[1,i].scatter(dataset[:, 0], dataset[:, 1], c=kmeans_k.labels_)
    ax[1,i].set_xlabel('x1')
    ax[1,i].set_ylabel('x2')
    ax[1,i].set_title('Clustering avec ' + str(best_K) + ' clusters')
fig.tight_layout()


**Conclusions :** 
- L'algorithme des k-moyennes permet d'obtenir un clustering satisfaisant pour le dataset avec quatre blobs bien séparés, y compris sans connaître le nombre idéal de clusters à l'avance, auquel cas le coefficient de silhouette permet de retrouver ce nombre idéal.
- En revanche, malgré l'optimisation du score de silhouette, cet algorithme ne permet pas d'obtenir de bons résultats pour les autres jeux de données, que ce soit pour les deux lunes imbriquées ou les deux cercles concentriques.

Nous allons donc maintenant essayer un autre algorithme de clustering et tester la performance de celui-ci pour la comparer au k-means. Nous allons nous limiter aux deux datasets pour lesquels l'algorithme k-means ne fonctionne pas.

### DBSCAN (Clustering par densité)

L'algorithme DBSCAN (Density-Based Spatial Clustering of Applications with Noise) fonctionne en deux temps :
- Toutes les observations suffisamment proches sont connectées entre elles.
- Les observations avec un nombre minimal de voisins connectés sont considérées comme des *core samples*, à partir desquelles les clusters sont étendues. **Toutes les observations suffisamment proche d'un *core sample* appartiennent au même cluster que celui-ci**. 

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

L'algorithme DBSCAN prend en entrée deux hyperparamètres :
- `eps` : la *taille du voisinage*, autrement dit la distance entre deux points de données en-dessous de laquelle un point est considéré à l'intérieur du voisinage de l'autre.

- `min_samples` : le nombre de voisins minimum pour qu'un point de données soit considéré comme un *core sample*.

In [ ]:
### DEBUT DE VOTRE CODE
# Initialisation de deux modèles DBSCAN
# avec les hyperparamètres eps=0.2, min_samples=2:
dbscan_moons = ...
dbscan_circles = ...

# Application aux données 
...
...
### FIN DE VOTRE CODE


L'attribut `.labels_` contient, pour chaque observation, le numéro du cluster auquel cette observation est assignée.

In [ ]:
print("Nombre d'étiquettes pour le dataset moons:", len(np.unique(dbscan_moons.labels_)))
print("Nombre d'étiquettes pour le dataset circles:", len(np.unique(dbscan_circles.labels_)))

Visualisons les clusters obtenus :

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

ax[0].scatter(moons[:, 0], moons[:, 1], c=dbscan_moons.labels_)
ax[0].set_title("Clustering DBSCAN (eps=0.2)")

ax[1].scatter(circles[:, 0], circles[:, 1], c=dbscan_circles.labels_)
ax[1].set_title("Clustering DBSCAN (eps=0.2)")

On peut voir ici que l'algorithme DBSCAN est capable d'identifier les deux clusters respectifs dans les deux datasets qui posaient problème à l'algorithme k-means. 

On peut noter également que nous n'avons pas eu besoin de renseigner un nombre de clusters à priori pour que l'algorithme identifie correctement le bon nombre de clusters. En revanche, l'algorithme est sensible aux deux hyperparamètre cités plus haut, ce que nous allons évaluer maintenant. 

#### Rôle du paramètre de taille de voisinage (`eps`)

Nous allons évaluer l'influence du paramètre `eps` sur le jeu de données des cercles concentriques (`circles`)

In [ ]:
### DEBUT DE VOTRE CODE
# Initialisation d'un clustering DBSCAN avec eps petit (ex, 0.05) et d'un autre avec eps grand (ex, 2.0):
dbscan_low = ...
dbscan_high = ...

# Application aux données 
...
...
### FIN DE VOTRE CODE

**Question :** Quel est le nombre de clusters obtenus dans nos deux modèles ? (Utiliser l'attribut `.labels`)

In [ ]:
### DEBUT DE VOTRE CODE
...
...
### FIN DE VOTRE CODE

In [ ]:
fig = plt.figure(figsize=(5, 5))

outliers = np.where(dbscan_low.labels_ == -1)[0]
plt.scatter(circles[outliers, 0], circles[outliers, 1], marker='*', color='red')

non_outliers = np.where(dbscan_low.labels_ != -1)[0]
plt.scatter(circles[non_outliers, 0], circles[non_outliers, 1], c=dbscan_low.labels_[non_outliers])
plt.title("Clustering DBSCAN (eps=0.05)")

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(circles[:, 0], circles[:, 1], c=dbscan_high.labels_)
plt.title("Clustering DBSCAN (eps=2.)")

#### Trouver eps avec le coefficient de silhouette

In [ ]:
print("Coefficient de silhouette pour DBSCAN (eps=0.2) : %.2f" % 
      metrics.silhouette_score(circles, dbscan_circles.labels_))

In [ ]:
eps_values = np.logspace(-3, 1, 40)
silhouettes = []

for eps in eps_values:
    dbscan_eps = cluster.DBSCAN(eps=eps, min_samples=2)
    dbscan_eps.fit(circles)
    if len(np.unique(dbscan_eps.labels_)) > 1: # nécessaire pour calculer le coeff de silhouette
        silhouettes.append(metrics.silhouette_score(circles, dbscan_eps.labels_))
    else:
        silhouettes.append(-1)

In [ ]:
plt.plot(eps_values, silhouettes)
plt.xscale("log")
plt.xlabel("eps (échelle log)")
plt.ylabel("silhouette")

In [ ]:
best_silhouette = np.max(silhouettes)
print("Coefficient de silhouette optimal : %.2f" % best_silhouette)
print("Eps correspondant : %.2f" % eps_values[silhouettes.index(best_silhouette)])

Voyons ce que donne le clustering obtenu en utilisant le paramètre `eps` avec lequel on obtient le meilleur coefficient de silhouette :

In [ ]:
best_eps = eps_values[silhouettes.index(best_silhouette)]

dbscan_best = cluster.DBSCAN(eps=best_eps, min_samples=2)
dbscan_best.fit(circles)

fig = plt.figure(figsize=(5, 5))
plt.scatter(circles[:, 0], circles[:, 1], c=dbscan_best.labels_)
plt.title("Clustering DBSCAN (eps=%.2f)" % best_eps)

**Question :**  Quel est le problème ici ? Le coefficient de silhouette est-il adapté à notre dataset ? 

### Index de Rand ajusté

L'index de Rand ajusté permet de **comparer un résultat de clustering avec des étiquettes**. Pour chaque paire d'observations nous regardons si elles se situent dans le même cluster ou non, dans le clustering prédit et réel. L'index prend des valeurs entre 0 (clustering aléatoire) et 1 (clustering parfait).

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html

__Question :__ Pourquoi ne pas utiliser une métrique d'évaluation de modèle de classification ici ?

In [ ]:
print("Index de Rand ajusté du K-means (K=2) : %.2f" % metrics.adjusted_rand_score(circles_labels, kmeans_circles.labels_))

In [ ]:
print("Index de Rand ajusté de dbscan (eps=0.2) : %.2f" % metrics.adjusted_rand_score(circles_labels, dbscan_circles.labels_))

## Bonus: Clustering sur les Manchots

Le but est ici de tester plusieurs méthodes non-supervisées de clustering sur un nouveau dataset, et de les comparer. Essayez les méthodes vues plus haut telles que [sklearn.cluster.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) ou [sklearn.cluster.DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). Vous pouvez aussi essayer d'autres méthodes telles que le mélange de Gaussiennes ([sklearn.mixture.GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)).

Quelles méthodes devraient mieux fonctionner selon vous ? Pourquoi ? 

Le jeu de données que nous vous proposons d'utiliser ici concerne différentes espèces de manchots et certaines caractéristiques physiques. Les 3 espèces sont : les manchots Adélie, les manchots papou (gentoo) et les manchots à jugulaire (chinstrap).

In [ ]:
palmerpenguins = pd.read_csv("data/penguins_data.csv")
palmerpenguins.head()

__Alternativement :__ Si vous avez besoin de télécharger le fichier (par exemple sur colab), décommentez les deux lignes suivantes :

In [ ]:
# !wget https://raw.githubusercontent.com/CBIO-mines/fml-dassault-systems/main/data/penguins_data.csv

# palmerpenguins = pd.read_csv("penguins_data.csv")

La première étape est de filtrer certaines données pour éviter les données manquantes (NA ou NaN).

In [ ]:
palmerpenguins = palmerpenguins[palmerpenguins['bill_depth_mm'].notna()]
palmerpenguins = palmerpenguins.reset_index(drop=True)

On va se concentrer ici uniquement sur deux caractéristiques de nos manchots : la longueur du bec et leur poids

In [ ]:
penguins_X = np.array(palmerpenguins[["bill_length_mm", "body_mass_g"]])

In [ ]:
from sklearn import preprocessing

Nos deux variables ne sont pas du tout à la même échelle, comme on peut le voir dans les données affichées plus haut. Il faut donc standardiser nos données.

In [ ]:
# standardisation (centrer-réduire)
penguins_X = preprocessing.StandardScaler().fit_transform(penguins_X)

In [ ]:
species_names, species_int = np.unique(palmerpenguins.species, return_inverse=True)
penguins_labels = species_int
species_names

Essayons d'abord de visualiser nos données sur un graphique :

In [ ]:
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels)
plt.xlabel("bill_length_mm (centrée-réduite)")
plt.ylabel("body_mass_g (centrée-réduite)")

A vous maintenant de tester différents algorithmes de clustering sur ces données, et d'en évaluer les performances. Pourrez-vous obtenir un clustering parfait ? 

En plus des algorithmes de clustering utilisés ci-dessus, vous pouvez essayer le mélange de Gaussiennes ([GaussianMixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)) par exemple, ou d'autres méthodes des modules `cluster` ou `mixture`.
Donnez pour chacun les coefficients de silhouette et indices de Rand ajusté que vous obtenez, ainsi qu'une visualisation du clustering obtenu. 

### KMeans

### DBSCAN

### Modèle de mélange gaussien 

Le modèle de mélange de gaussiennes cherche à **optimiser les paramètres d'un nombre fini de gaussiennes** aux données. 

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html